In [52]:
import csv

In [53]:
myData = "data/NationalFile_20200101.txt"
myOutput = "data/love-and-valentine.geojson"

In [54]:
# Always include home
loveCentral = """{"type": "Feature",
                 "properties": {
                    "FEATURE_NAME": "❤️",
                    "FEATURE_CLASS": "Home is where the heart is.",
                    "STATE_ALPHA": "KY, Y'ALL",
                    "ELEV_IN_FT": "-9999" },
                "geometry": {"type": "Point","coordinates":[-84.5042992, 38.0387101]}}
                ]}
                """
# Start GeoJSON template
templateStart = """{
    "type": "FeatureCollection",
    "features": [
"""

with open(myData, encoding = 'utf-8') as csvFile:
    #open empty file to write output
    with open(myOutput, "w", encoding = 'utf-8', newline = "") as geoJson:
        # create reader object 
        reader = csv.reader(csvFile, delimiter = '|')
        geoJson.write(templateStart)
        for row in reader:
            # Search for standalone "love" or "valentine"
            if " love " in row[1].lower() or " valentine " in row[1].lower():
                # Format GeoJSON feature
                feature = """
                {"type": "Feature",
                 "properties": {"""
                feature += f"""
                    "FEATURE_NAME": "{row[1]}",
                    "FEATURE_CLASS": "{row[2]}",
                    "STATE_ALPHA": "{row[3]}",
                    "ELEV_IN_FT": "{row[-4]}"
                """
                feature += """},
                "geometry": {"type": "Point","coordinates":"""
                feature += f'[{float(row[-10])},{float(row[-11])}]'
                feature += '}},\n'
                # Write feature
                geoJson.write(feature)
        # Write home
        geoJson.write(loveCentral)